# Chatbot Faceoff

This notebook enables a faceoff between two models chatting with one another.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import gradio as gr

In [ ]:
load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
LLAMA_MODEL = "llama3.2"
GEMINI_MODEL = "gemini-1.5-flash"
llama = OpenAI(base_url='http://localhost:11434/v1', api_key='dummy')
gemini = OpenAI(base_url='https://generativelanguage.googleapis.com/v1beta/openai/', api_key=google_api_key)

In [ ]:
# Let's make a conversation between llama and gemini

llama_system = "You are a narcissistic chatbot who speaks in the style of Donald Trump. Keep your responses brief, no more than 200 words."

gemini_system = "You are a very humorous chatbot. You state everything in a humorous way and try to elicit laughs from your interactions. Keep your responses brief, no more than 200 words."

In [ ]:
llama_messages = []
gemini_messages = []

In [ ]:
def call_llama():
    messages = [{"role": "system", "content": llama_system}]
    for llama_message, gemini_message in zip(llama_messages, gemini_messages):
        messages.append({"role": "assistant", "content": llama_message})
        messages.append({"role": "user", "content": gemini_message})
    print(messages)
    completion = llama.chat.completions.create(
        model=LLAMA_MODEL,
        messages=messages,
    )
    message = completion.choices[0].message.content
    llama_messages.append(message)
    return message

In [ ]:
def call_gemini():
    messages = [{"role": "system", "content": gemini_system}]
    for llama_message, gemini_message in zip(llama_messages, gemini_messages):
        messages.append({"role": "user", "content": llama_message})
        messages.append({"role": "assistant", "content": gemini_message})
    messages.append({"role": "user", "content": llama_messages[-1]})
    print(messages)
    completion = gemini.chat.completions.create(
        model=GEMINI_MODEL,
        messages=messages,
    )
    message = completion.choices[0].message.content
    gemini_messages.append(message)
    return message

In [ ]:
llama_messages = ['Hi there']
gemini_messages = []

with gr.Blocks() as ui:
    gr.Markdown("Chatbot Faceoff")
    messages = gr.State([])
    model = ""

    def add_message(previous_messages):
        global model
        content = ""

        if len(previous_messages) == 0:
            content = "Hi there"
            model = "llama"
        else:
            if model == 'llama':
                model = 'gemini'
                content = call_gemini()
            else:
                model = 'llama'
                content = call_llama()
        
        message = {'model': model, 'content': content}
        messages = previous_messages + [message]
        return messages

    gr.Button("Chat").click(add_message, messages, messages)

    @gr.render(inputs=messages)
    def render_messages(messages):
        for i in reversed(range(len(messages))):
            gr.Textbox(key=i, label=messages[i]['model'], value=messages[i]['content'])
        
ui.launch(inbrowser=True)